<a href="https://colab.research.google.com/github/vdslab/aggregated-document-dendrogram/blob/main/ipynb/judgit/reviewsheet_top2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
!pip install mojimoji
!pip install gensim
!pip install jaconv
!pip install ja_stopword_remover

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [2]:
!pip3 install neologdn
!pip3 install top2vec[sentence_transformers]
!pip3 install gensim
!pip3 install tensorflow
!pip3 install torch
!pip3 install ipywidgets
!pip3 install japanize-matplotlib
!pip3 install fugashi
!pip3 install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.8 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172954 sha256=71514efc266b978d94e7168197f95944d6611f7b4066eb0aee70aa09207315c8
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 11.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 24.1 MB 1.8 MB/s 
     |████████████████████████████████| 88 kB 4.4 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.5 MB/s 
     |███████████████████████████

In [ ]:
import torch
from transformers import BertModel
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [ ]:
import json
import pandas as pd
with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/RS21.json', 'r', encoding='utf8')as f:
    df = pd.read_json(f)

In [ ]:
documents = []
for data in df['Abstract']:
    if data !='':
        documents.append(data)

In [ ]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import MeCab
def tokenize(text):
    tagger = MeCab.Tagger("mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
    lines = tagger.parse(' '.join(text.split())).splitlines()
    words = []
    for line in lines:
        chunk = line.split('\t')
        tmp = chunk[-1].split(',')
        if tmp[0] != 'EOS':
            if tmp[6] != '*':
                words.append(tmp[6])
            else:
                words.append(chunk[0])
    return words

In [ ]:
import neologdn
import re

corpus = []
for data in documents:
    try:
        # words = tokenize(re.sub(r'[0-9]+', "0", neologdn.normalize(data[k][2])))
        corpus.append(neologdn.normalize(data))
    except:
        pass

In [ ]:
import transformers
from top2vec import Top2Vec
import sentence_transformers
import random

transformers.BertTokenizer = transformers.BertJapaneseTokenizer
model = Top2Vec(documents=corpus,
                    embedding_model="sonoisa/sentence-bert-base-ja-mean-tokens-v2",
                    tokenizer=tokenize,
                    split_documents=True,
                    document_chunker='random'
                )

2022-06-06 08:32:46,186 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-06 08:33:00,194 - top2vec - INFO - Downloading sonoisa/sentence-bert-base-ja-mean-tokens-v2 model
2022-06-06 08:33:02,945 - top2vec - INFO - Creating joint document/word embedding
2022-06-06 08:54:45,445 - top2vec - INFO - Creating lower dimension embedding of documents
2022-06-06 08:55:00,114 - top2vec - INFO - Finding dense areas of documents
2022-06-06 08:55:00,282 - top2vec - INFO - Finding topics


In [ ]:
model.topic_sizes

0    4345
1     120
2     101
3      79
dtype: int64

In [ ]:
model.topic_sizes

0    4273
1      62
2      60
3      59
4      58
5      58
6      56
7      19
dtype: int64

In [ ]:
model.topic_words[0]

array(['本事業は', '定額補助', '補助率', '本事業ては', '実施主体', '委託', '補助', '具体的には', '定額',
       '分析', 'に基つき', '年度', 'について', '以下', 'なお', '条第', 'また', '平成', 'において',
       '令和', '分の', '都道府県', '万円', '市町村', '以内', 'という', 'さらに', '昭和'],
      dtype='<U5')

In [ ]:
model = load_model('cl-tohoku/bert-base-japanese-whole-word-masking')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
